<a href="https://colab.research.google.com/github/Ruqyai/Convert-YouTube-to-PPTX/blob/main/convert_youtube_to_pptx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
sudo apt-get update
sudo apt-get install tesseract-ocr
sudo apt-get install libtesseract-dev
pip install pytube
pip install pytesseract
pip install python-pptx
pip install reportlab


In [ ]:
from pytube import YouTube
import cv2
import pytesseract
from PIL import Image
from pptx import Presentation
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os

In [ ]:
# Download YouTube video
video_url = "https://youtu.be/.."
youtube = YouTube(video_url)
video_stream = youtube.streams.get_highest_resolution()
video_path = 'videos/video.mp4'
video_stream.download(output_path='videos', filename='video.mp4')

In [ ]:
# Create the 'frames' directory if it doesn't exist
os.makedirs('frames', exist_ok=True)

# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)
frame_count = 0

# Create a PowerPoint presentation
ppt = Presentation()


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      break
    frame_count += 1
    image_path = f'frames/frame_{frame_count}.png'
    cv2.imwrite(image_path, frame)

    # Perform OCR on the image
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)

    # Create a new slide
    slide = ppt.slides.add_slide(ppt.slide_layouts[5])

    # Add image to the slide
    left = top = 0  # You might need to adjust these coordinates
    pic = slide.shapes.add_picture(image_path, left, top, height=ppt.slide_height)

    # Add extracted text to the slide
    left_text = 0  # You might need to adjust this coordinate
    top_text = pic.top + pic.height + 10  # Adjust the vertical distance from the image
    text_box = slide.shapes.add_textbox(left_text, top_text, pic.width, 100)
    text_frame = text_box.text_frame
    text_frame.text = text

cap.release()


In [ ]:
# Save the PowerPoint presentation
ppt.save('output.pptx')
ppt.save('output.pdf')
